In [84]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [85]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [86]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [87]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Extraction

In [88]:
cliente = pd.read_sql_table('dim_cliente', etl_conn)
mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
fecha = pd.read_sql_table('dim_fecha', etl_conn)
servicio = pd.read_sql_table("mensajeria_servicio",url_db)

In [89]:
total_nan = servicio.isna().sum()
print(total_nan)

id                                    0
descripcion                           0
nombre_solicitante                    0
fecha_solicitud                       0
hora_solicitud                        0
fecha_deseada                         6
hora_deseada                          0
nombre_recibe                         0
telefono_recibe                       0
descripcion_pago                    260
ida_y_regreso                         0
activo                                0
novedades                           336
cliente_id                            0
destino_id                            0
mensajero_id                        727
origen_id                             0
tipo_pago_id                          0
tipo_servicio_id                      0
tipo_vehiculo_id                      0
usuario_id                            0
prioridad                             0
ciudad_destino_id                     0
ciudad_origen_id                      0
hora_visto_por_mensajero          28428


In [90]:
print(len(servicio))

28430


In [91]:
servicio.head(5)

,id,descripcion,nombre_solicitante,fecha_solicitud,hora_solicitud,fecha_deseada,hora_deseada,nombre_recibe,telefono_recibe,descripcion_pago,...,ciudad_origen_id,hora_visto_por_mensajero,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado
0,34,recoger muestras a las 10 am,chat_GPT,2023-10-26,09:46:03,2023-10-26,09:46:03,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
1,35,Recogervmx a las,chat_GPT,2023-10-26,11:18:14,2023-10-26,11:18:14,Gemini,100,,...,1,None,,,NaN,NaN,False,False,True,
2,36,recoger mx a las 9 am,chat_GPT,2023-10-28,19:21:01,2023-10-30,19:21:01,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
3,41,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:09,2023-11-07,09:46:09,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,
4,42,recoger mx a las 11 am,chat_GPT,2023-11-07,09:46:10,2023-11-07,09:46:10,Gemini,100,,...,1,None,,,NaN,NaN,False,True,False,


## Transformation

In [92]:
fecha_unique = fecha[["day_str", "key_fecha"]].drop_duplicates(subset=["day_str"])
servicio["dia"] = servicio["fecha_solicitud"].dt.day_name().str.lower()
servicio = servicio[["id", "dia", "cliente_id", "mensajero_id"]]
servicio.fillna(-1, inplace=True)

# Asegurar que los valores en la columna "dia" coincidan antes de realizar el merge
servicio["dia"] = servicio["dia"].str.capitalize() # Capitalizar los valores en la columna "dia" del dataframe "servicio"
hecho_servicio_dia = pd.merge(servicio, fecha_unique[["day_str", "key_fecha"]], left_on="dia", right_on="day_str", how="left")
hecho_servicio_dia.head(5)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha
0,34,Thursday,5,-1.0,Thursday,3
1,35,Thursday,5,7.0,Thursday,3
2,36,Saturday,5,-1.0,Saturday,5
3,41,Tuesday,5,-1.0,Tuesday,1
4,42,Tuesday,5,-1.0,Tuesday,1


In [93]:
hecho_servicio_dia = pd.merge(hecho_servicio_dia, cliente[["cliente_id", "key_cliente"]], left_on="cliente_id", right_on="cliente_id", how="left")
hecho_servicio_dia.head(5)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha,key_cliente
0,34,Thursday,5,-1.0,Thursday,3,7
1,35,Thursday,5,7.0,Thursday,3,7
2,36,Saturday,5,-1.0,Saturday,5,7
3,41,Tuesday,5,-1.0,Tuesday,1,7
4,42,Tuesday,5,-1.0,Tuesday,1,7


In [94]:
mensajero.rename(columns={'id':'mensajero_id'}, inplace=True)
hecho_servicio_dia = hecho_servicio_dia.merge(mensajero[["mensajero_id", "key_mensajero"]], left_on="mensajero_id", right_on="mensajero_id", how="left")
hecho_servicio_dia.fillna(-1, inplace=True)
hecho_servicio_dia.head(5)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha,key_cliente,key_mensajero
0,34,Thursday,5,-1.0,Thursday,3,7,-1.0
1,35,Thursday,5,7.0,Thursday,3,7,13.0
2,36,Saturday,5,-1.0,Saturday,5,7,-1.0
3,41,Tuesday,5,-1.0,Tuesday,1,7,-1.0
4,42,Tuesday,5,-1.0,Tuesday,1,7,-1.0


In [95]:
print(len(hecho_servicio_dia))

28430


## Cálculo del atributo solicitudes_por_dia

In [96]:
# Contar solicitudes por cada 'dia'
solicitudes_por_dia = hecho_servicio_dia.groupby("dia").size().reset_index(name="solicitud_por_dia")
# Hacer merge para agregar la columna 'solicitud_por_dias' al DataFrame original
hecho_servicio_dia = hecho_servicio_dia.merge(solicitudes_por_dia, on="dia", how="left")
hecho_servicio_dia.head(5)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha,key_cliente,key_mensajero,solicitud_por_dia
0,34,Thursday,5,-1.0,Thursday,3,7,-1.0,5161
1,35,Thursday,5,7.0,Thursday,3,7,13.0,5161
2,36,Saturday,5,-1.0,Saturday,5,7,-1.0,2481
3,41,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398
4,42,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398


In [97]:
print(len(hecho_servicio_dia))

28430


## Cálculo del atributo solicitudes_por_dia_cliente

In [98]:
# Contar solicitudes por cada 'key_fecha' y 'key_cliente' 
solicitud_por_dia_cliente = hecho_servicio_dia.groupby(["key_fecha","key_cliente"]).size().reset_index(name="solicitud_por_dia_cliente")
# Hacer merge para agregar la columna 'solicitud_por_dia' al DataFrame original
hecho_servicio_dia = hecho_servicio_dia.merge(solicitud_por_dia_cliente, on=["key_cliente", "key_fecha"], how="left")
hecho_servicio_dia.head(5)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha,key_cliente,key_mensajero,solicitud_por_dia,solicitud_por_dia_cliente
0,34,Thursday,5,-1.0,Thursday,3,7,-1.0,5161,887
1,35,Thursday,5,7.0,Thursday,3,7,13.0,5161,887
2,36,Saturday,5,-1.0,Saturday,5,7,-1.0,2481,442
3,41,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398,933
4,42,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398,933


In [99]:
print(len(hecho_servicio_dia))

28430


## Cálculo del atributo solicitudes_por_mensajero

In [100]:
# Contar solicitudes por cada 'key_fecha' y 'key_mensajero'
cantidad_solicitud_por_mensajero = hecho_servicio_dia.groupby(["key_fecha","key_mensajero"]).size().reset_index(name="cantidad_solicitud_por_dia_mensajero")
# Hacer merge para agregar la columna 'solicitud_por_dia' al DataFrame original
hecho_servicio_dia = hecho_servicio_dia.merge(cantidad_solicitud_por_mensajero, on=["key_fecha","key_mensajero"], how="left")
hecho_servicio_dia.head(6)

,id,dia,cliente_id,mensajero_id,day_str,key_fecha,key_cliente,key_mensajero,solicitud_por_dia,solicitud_por_dia_cliente,cantidad_solicitud_por_dia_mensajero
0,34,Thursday,5,-1.0,Thursday,3,7,-1.0,5161,887,127
1,35,Thursday,5,7.0,Thursday,3,7,13.0,5161,887,21
2,36,Saturday,5,-1.0,Saturday,5,7,-1.0,2481,442,63
3,41,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398,933,135
4,42,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398,933,135
5,43,Tuesday,5,-1.0,Tuesday,1,7,-1.0,5398,933,135


## Eliminar filas no necesarias

In [101]:
hecho_servicio_dia.drop(columns=["id","cliente_id","dia","mensajero_id","day_str"], inplace=True)
hecho_servicio_dia.head(5)

,key_fecha,key_cliente,key_mensajero,solicitud_por_dia,solicitud_por_dia_cliente,cantidad_solicitud_por_dia_mensajero
0,3,7,-1.0,5161,887,127
1,3,7,13.0,5161,887,21
2,5,7,-1.0,2481,442,63
3,1,7,-1.0,5398,933,135
4,1,7,-1.0,5398,933,135


## Eliminar Datos Duplicados

In [102]:
hecho_servicio_dia_sin_duplicados = hecho_servicio_dia.drop_duplicates()
len(hecho_servicio_dia_sin_duplicados)

944

In [103]:
hecho_servicio_dia_sin_duplicados

,key_fecha,key_cliente,key_mensajero,solicitud_por_dia,solicitud_por_dia_cliente,cantidad_solicitud_por_dia_mensajero
0,3,7,-1.0,5161,887,127
1,3,7,13.0,5161,887,21
2,5,7,-1.0,2481,442,63
3,1,7,-1.0,5398,933,135
6,3,6,24.0,5161,14,92
...,...,...,...,...,...,...
28249,5,11,18.0,2481,1187,5
28250,5,11,4.0,2481,1187,1
28314,0,7,13.0,4308,673,4
28324,1,7,0.0,5398,933,1


## Loading

In [104]:
hecho_servicio_dia_sin_duplicados.to_sql("hecho_servicio_dia", etl_conn, if_exists="replace", index_label="key_solicitud_servicio") 

944